<a href="https://colab.research.google.com/github/batlab-art/Introductions/blob/master/examples/OpenCV_Pointillism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [31]:
!git clone https://github.com/matteo-ronchetti/Pointillism.git

fatal: destination path 'Pointillism' already exists and is not an empty directory.


In [32]:
!pip install kora
from kora import console
console.start()  # and click link

Console URL: https://teleconsole.com/s/d9f2a46fdc8b9c9e93050941f8618a1543b26c47


In [ ]:
!pip install icrawler

In [34]:
import math
import progressbar

from Pointillism.pointillism import *

# Fetch Images
### BingImageCrawler:
**Type** – “photo”, “clipart”, “linedrawing”, “transparent”, “animated"
**Color** – “color”, “blackandwhite”, “red”, “orange”, “yellow”, “green”, “teal”, “blue”, “purple”, “pink”, “white”, “gray”, “black”, “brown
**Size** – “large”, “medium”, “small” or larger than a given size (e.g. “>640x480”).
**License** – “creativecommons”, “publicdomain”, “noncommercial”, “commercial”, “noncommercial,modify”, “commercial,modify”.
**Layout** – “square”, “wide”, “tall”.
**People** – “face”, “portrait”.
**Date** – “pastday”, “pastweek”, “pastmonth”, “pastyear”.

https://icrawler.readthedocs.io/en/latest/builtin.html

In [ ]:
!rm /content/download/*

from icrawler.builtin import BingImageCrawler

# הגדרת משתנים עבור פרמטרים לחיפוש תמונות
#

keywordText = 'wind shade range'  #@param {type: "string"}
num_images =   8 #@param {type: "number"}
directory = 'download'  #@param {type: "string"}

# ביצוע חיפוש והורדת תמונות מבינג
#

bing_crawler = BingImageCrawler(storage={'root_dir': directory})
bing_crawler.crawl(keyword=keywordText, max_num=num_images)

output_directory = '/content/' + directory + '/'

# Pointillize function

In [ ]:
import matplotlib.pyplot as plt
import cv2

def Pointillize (filenameAndPath, stroke_scale = 0, gradient_smoothing_radius = 0, limit_image_size = 0, palette_size = 10):

  img = cv2.imread(filenameAndPath)

  if limit_image_size > 0:
    img = limit_size(img, limit_image_size)

  if stroke_scale == 0:
      stroke_scale = int(math.ceil(max(img.shape) / 1000))
      print("Automatically chosen stroke scale: %d" % stroke_scale)
  else:
     stroke_scale = stroke_scale

  if gradient_smoothing_radius == 0:
      gradient_smoothing_radius = int(round(max(img.shape) / 50))
      print("Automatically chosen gradient smoothing radius: %d" % gradient_smoothing_radius)
  else:
      gradient_smoothing_radius = gradient_smoothing_radius

  # convert the image to grayscale to compute the gradient
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  print("Computing color palette...")
  palette = ColorPalette.from_image(img, palette_size)

  print("Extending color palette...")
  palette = palette.extend([(0, 50, 0), (15, 30, 0), (-15, 30, 0)])

  # display the color palette
  #plt.imshow(palette.to_image())
  # cv2.imshow("palette", palette.to_image())
  # cv2.waitKey(200)

  print("Computing gradient...")
  gradient = VectorField.from_gradient(gray)

  print("Smoothing gradient...")
  gradient.smooth(gradient_smoothing_radius)

  print("Drawing image...")
  # create a "cartonized" version of the image to use as a base for the painting
  res = cv2.medianBlur(img, 11)
  # define a randomized grid of locations for the brush strokes
  grid = randomized_grid(img.shape[0], img.shape[1], scale=3)
  batch_size = 10000

  bar = progressbar.ProgressBar()
  for h in bar(range(0, len(grid), batch_size)):
    # get the pixel colors at each point of the grid
    pixels = np.array([img[x[0], x[1]] for x in grid[h:min(h + batch_size, len(grid))]])
    # precompute the probabilities for each color in the palette
    # lower values of k means more randomnes
    color_probabilities = compute_color_probabilities(pixels, palette, k=9)

    for i, (y, x) in enumerate(grid[h:min(h + batch_size, len(grid))]):
        color = color_select(color_probabilities[i], palette)
        angle = math.degrees(gradient.direction(y, x)) + 90
        length = int(round(stroke_scale + stroke_scale * math.sqrt(gradient.magnitude(y, x))))

        # draw the brush stroke
        cv2.ellipse(res, (x, y), (length, stroke_scale), angle, 0, 360, color, -1, cv2.LINE_AA)

  return res, palette.to_image()


# Process Images

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# List Files
# os.listdir returns a list of all files in output_directory 
# the list contains only filenames - to access a file from the list you need to
# concat output_directory + inputFiles[n]
# 

inputFiles = os.listdir(output_directory)
resultsFolder = '/content/out/'

if not os.path.exists(resultsFolder):
  os.mkdir(resultsFolder)
else:
  filelist = os.listdir(resultsFolder)
  for f in filelist:
    os.remove(os.path.join(resultsFolder, f))

for file in inputFiles:
  src = cv2.imread(output_directory + file)
  stroke_scale = 0 # random.randint(1, 4)
  gradient_smoothing_radius = 0 
  limit_image_size = 0
  palette_size = 10 # random.randint(6, 10)
  result, pallete = Pointillize (output_directory + file)
  cv2.imwrite(resultsFolder + file, result)


In [ ]:
from google.colab.patches import cv2_imshow

resultsFolder = '/content/out/'
outFiles = os.listdir(resultsFolder)

# Display
for i in range(len(outFiles)):
    cv2_imshow(cv2.imread(resultsFolder + outFiles[i]))
  #  cv2_imshow(cv2.cvtColor(cv2.imread(resultsFolder + outFiles[i]),cv2.COLOR_BGR2RGB))
